<a href="https://colab.research.google.com/github/Anam2291/Deep_Learning_Projects/blob/main/Data_Augmentation_Cats_Vs_Dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setting Up the Kaggle API**

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/


# **Downloading the Dataset**

In [18]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


# **Extracting the Dataset**

This extracts the dataset into the /content directory.

In [19]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,         # Normalize pixel values
    rotation_range=40,       # Randomly rotate images
    width_shift_range=0.2,   # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Random zoom
    horizontal_flip=True,    # Flip images horizontally
    fill_mode='nearest'      # Fill any new pixels
)



In [ ]:
# Apply to training data
train_generator = train_datagen.flow_from_directory(
    '/content/train',        # Path to training data
    target_size=(150, 150),  # Resize all images to 150x150
    batch_size=32,
    class_mode='binary'      # Use 'binary' for binary classification
)

# For validation, use rescaling only (no augmentation)
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(
    '/content/validation',   # Path to validation data
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)


# **Training Model Using model.fit**

We'll use a simple Convolutional Neural Network (CNN) for binary classification (Cats vs. Dogs).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])


# **Compile the Model**

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',         # Loss function for binary classification
    optimizer=RMSprop(learning_rate=1e-4),
    metrics=['accuracy']                # Track accuracy during training
)


# **Train the Model**

In [ ]:
history = model.fit(
    train_generator,                   # Training data generator
    steps_per_epoch=100,               # Number of training batches per epoch
    epochs=10,                         # Number of epochs to train
    validation_data=validation_generator,  # Validation data generator
    validation_steps=50                # Number of validation batches per epoch
)


# **Visualize Training and Validation Performance**

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()
